<a href="https://colab.research.google.com/github/advait1987/tf_gendervoiceclassify/blob/master/Gender_Voice_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
!pip install -q tensorflow==2.0.0-alpha0
import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split

     |████████████████████████████████| 79.9MB 312kB/s 
     |████████████████████████████████| 3.0MB 38.5MB/s 
     |████████████████████████████████| 61kB 20.6MB/s 
     |████████████████████████████████| 419kB 46.6MB/s 


In [2]:
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../tmp"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

In [0]:
url = 'https://github.com/advait1987/tf_gendervoiceclassify/raw/master/voice.csv'
dataframe = pd.read_csv(url)

## **Explore**

In [4]:
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,male
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,male


In [5]:
from sklearn.preprocessing import LabelEncoder
gender_encoder = LabelEncoder()
dataframe['label'] = gender_encoder.fit_transform(dataframe['label'])
dataframe.head(2)

,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
0,0.059781,0.064241,0.032027,0.015071,0.090193,0.075122,12.863462,274.402906,0.893369,0.491918,0.0,0.059781,0.084279,0.015702,0.275862,0.007812,0.007812,0.007812,0.000000,0.000000,1
1,0.066009,0.067310,0.040229,0.019414,0.092666,0.073252,22.423285,634.613855,0.892193,0.513724,0.0,0.066009,0.107937,0.015826,0.250000,0.009014,0.007812,0.054688,0.046875,0.052632,1


In [6]:
print("Total number of samples: {}".format(dataframe.shape[0]))
print("Number of male: {}".format(dataframe[dataframe.label == 1].shape[0]))
print("Number of female: {}".format(dataframe[dataframe.label == 0].shape[0]))

Total number of samples: 3168
Number of male: 1584
Number of female: 1584


Split Data

In [7]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.2)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')
print (train.shape)
train.head()

2027 train examples
507 validation examples
634 test examples
(2027, 21)


,meanfreq,sd,median,Q25,Q75,IQR,skew,kurt,sp.ent,sfm,mode,centroid,meanfun,minfun,maxfun,meandom,mindom,maxdom,dfrange,modindx,label
2896,0.212141,0.043329,0.212018,0.192473,0.242610,0.050137,1.604483,5.042737,0.881385,0.308087,0.206495,0.212141,0.151118,0.047013,0.279070,1.635000,0.023438,7.617188,7.593750,0.145259,0
1556,0.162938,0.063401,0.155755,0.110694,0.223837,0.113143,3.126004,15.665239,0.911939,0.425609,0.115265,0.162938,0.110186,0.043668,0.222222,0.245117,0.034180,0.781250,0.747070,0.262092,1
2967,0.166562,0.057376,0.159473,0.144891,0.198603,0.053711,3.008209,13.797853,0.910222,0.492689,0.159842,0.166562,0.141490,0.019531,0.227273,0.512288,0.004883,1.362305,1.357422,0.360911,0
1447,0.178306,0.062549,0.178832,0.136423,0.235547,0.099124,1.178621,4.001226,0.923416,0.448994,0.159927,0.178306,0.112051,0.047151,0.279070,0.947368,0.023438,5.390625,5.367188,0.117647,1
1531,0.135390,0.079694,0.116456,0.100607,0.213847,0.113240,8.887654,152.919703,0.898122,0.462075,0.000000,0.135390,0.101600,0.015936,0.258065,0.030331,0.007812,0.125000,0.117188,0.395960,1


**Create an input pipeline using tf.data**
This will enable us to use feature columns as a bridge to map from the columns in the Pandas dataframe to features 
used to train the model. If we were working with a very large CSV file (so large that it does not fit into memory), we would use tf.data to read it from disk directly. That is not covered in this tutorial.

In [0]:
# A utility method to create a tf.data dataset from a Pandas Dataframe
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('label')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

batch_size = 5 # A small batch sized is used for demonstration purposes
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [9]:
# Test input pipeline
for feature_batch, label_batch in train_ds.take(1):
  print('Every feature:', list(feature_batch.keys()))
  print('A batch of ages:', feature_batch['median'])
  print('A batch of targets:', label_batch )

Every feature: ['meanfreq', 'sd', 'median', 'Q25', 'Q75', 'IQR', 'skew', 'kurt', 'sp.ent', 'sfm', 'mode', 'centroid', 'meanfun', 'minfun', 'maxfun', 'meandom', 'mindom', 'maxdom', 'dfrange', 'modindx']
A batch of ages: tf.Tensor([0.20564315 0.19573334 0.18781085 0.1948538  0.19025041], shape=(5,), dtype=float32)
A batch of targets: tf.Tensor([0 1 0 1 1], shape=(5,), dtype=int32)


**Choose which columns to use**

In [0]:
feature_columns = []

# numeric cols
for header in list(feature_batch.keys()):
  feature_columns.append(feature_column.numeric_column(header))

# bucketized cols
#age_buckets = feature_column.bucketized_column(age, boundaries=[18, 25, 30, 35, 40, 45, 50, 55, 60, 65])
#feature_columns.append(age_buckets)

# indicator cols
#thal = feature_column.categorical_column_with_vocabulary_list(
#      'thal', ['fixed', 'normal', 'reversible'])
#thal_one_hot = feature_column.indicator_column(thal)
#feature_columns.append(thal_one_hot)

# embedding cols
#thal_embedding = feature_column.embedding_column(thal, dimension=8)
#feature_columns.append(thal_embedding)

# crossed cols
#crossed_feature = feature_column.crossed_column([age_buckets, thal], hash_bucket_size=1000)
#crossed_feature = feature_column.indicator_column(crossed_feature)
#feature_columns.append(crossed_feature)

**Create feature layer**

In [0]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)
#Earlier, we used a small batch size to demonstrate how feature columns worked. We create a new input pipeline with a larger batch size.
batch_size = 32
train_ds = df_to_dataset(train, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

**Create, compile, and train the model**

In [44]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('acc')>0.95):
      print (logs.get('acc'))
      print("\nReached 95% accuracy so cancelling training!")
      self.model.stop_training = True

callbacks = myCallback()   

model = tf.keras.Sequential([
  feature_layer,
  #tf.keras.layers.Conv1D(filters=5, kernel_size=5, input_shape=(None,20), activation='relu'),
  #tf.keras.layers.MaxPooling2D(2,2),
  #tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(128, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(32, activation='relu'),
  tf.keras.layers.Dense(1, activation='sigmoid')
])


model.compile(optimizer=tf.keras.optimizers.Adam(0.001),
              loss='binary_crossentropy',
              metrics=['acc'])

model.fit(train_ds,
          validation_data=val_ds,
          epochs=50,
          callbacks=[callbacks])

#model.summary()

Epoch 1/50
64/64 [==============================] - 2s 28ms/step - loss: 0.8309 - acc: 0.5279 - val_loss: 0.6566 - val_acc: 0.5365
Epoch 2/50
64/64 [==============================] - 2s 24ms/step - loss: 0.6872 - acc: 0.5589 - val_loss: 0.6309 - val_acc: 0.5858
Epoch 3/50
64/64 [==============================] - 2s 24ms/step - loss: 0.7269 - acc: 0.6242 - val_loss: 0.6256 - val_acc: 0.6568
Epoch 4/50
64/64 [==============================] - 2s 25ms/step - loss: 0.6261 - acc: 0.6355 - val_loss: 0.5669 - val_acc: 0.6982
Epoch 5/50
64/64 [==============================] - 2s 26ms/step - loss: 0.6791 - acc: 0.6941 - val_loss: 0.5888 - val_acc: 0.7140
Epoch 6/50
64/64 [==============================] - 2s 26ms/step - loss: 0.6111 - acc: 0.6784 - val_loss: 0.5266 - val_acc: 0.7377
Epoch 7/50
64/64 [==============================] - 2s 26ms/step - loss: 0.6137 - acc: 0.7135 - val_loss: 0.5205 - val_acc: 0.7337
Epoch 8/50
64/64 [==============================] - 2s 26ms/step - loss: 0.5483 - a

**Accuracy**

In [45]:
loss, accuracy = model.evaluate(test_ds)
print("Accuracy", accuracy)

20/20 [==============================] - 0s 12ms/step - loss: 0.1265 - acc: 0.9558
Accuracy 0.95583594


**Decision Tree for comparison**